# Data Cleaning - ARTG Truck Processing (NEW DATASET)

**Dataset Baru - Improved Structure:**
- ✅ SLOT, ROW, TIER sudah terpisah sebagai kolom berbeda
- ✅ Tidak perlu parsing complex
- ✅ Filter BLOK 7 (tidak diprediksi)

**Cleaning Steps:**
1. Load data (kolom sudah terstruktur baik)
2. Remove BLOK 7 (7A, 7B, 7C, dll)
3. Validate slot/row/tier ranges
4. Remove duplicates & missing values
5. Export clean CSV

**Note:**
- Slot 102-103 di blok 3Z adalah VALID
- Data anomali (11305, 10203) berasal dari blok 7 → akan di-filter

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

✅ Libraries imported successfully!
Pandas version: 2.2.2
Numpy version: 1.26.4


## 2. Load Dataset

In [2]:
df = pd.read_csv('../Data/src/gatein_out_2bulan.csv', delimiter=';')

print("=" * 80)
print("DATASET LOADED")
print("=" * 80)
print(f"Shape: {df.shape}")
print(f"Rows: {len(df):,}")
print(f"Columns: {df.shape[1]}")
print(f"\nColumns:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

DATASET LOADED
Shape: (122540, 22)
Rows: 122,540
Columns: 22

Columns:
   1. TRUCK_ID
   2. JOB_TYPE
   3. CONTAINER_KEY
   4. CONTAINER_SIZE
   5. CTR_STATUS
   6. CONTAINER_TYPE
   7. CONTAINER_NO
   8. GATE_IN
   9. PARKING_IN
  10. PARKING_OUT
  11. BUFFER_IN
  12. STACK_TIME
  13. SLOT
  14. ROW
  15. TIER
  16. STACK
  17. BUFFER_OUT
  18. GATE_OUT
  19. GATE_IN_STACK
  20. GATE_IN_OUT
  21. PARK_IN_OUT
  22. GATE_IN_PARK_OUT


In [3]:
# Preview data
print("\n" + "=" * 80)
print("SAMPLE DATA (First 10 rows)")
print("=" * 80)
df.head(10)


SAMPLE DATA (First 10 rows)


,TRUCK_ID,JOB_TYPE,CONTAINER_KEY,CONTAINER_SIZE,CTR_STATUS,CONTAINER_TYPE,CONTAINER_NO,GATE_IN,PARKING_IN,PARKING_OUT,...,SLOT,ROW,TIER,STACK,BUFFER_OUT,GATE_OUT,GATE_IN_STACK,GATE_IN_OUT,PARK_IN_OUT,GATE_IN_PARK_OUT
0,B9551UWV,Receiving,7654527,40,FCL,DRY,WHSU8333154,2025-10-04 00:01:19.000,2025-10-04 00:02:04.000,2025-10-04 00:03:48.000,...,26,06,3,5B,2025-10-04 00:16:38.000,2025-10-04 00:19:03.000,10.47,17.73,1.73,2.48
1,G8470OE,Receiving,7654517,40,FCL,DRY,UETU8373785,2025-10-04 00:01:21.000,2025-10-04 00:02:04.000,2025-10-04 00:04:24.000,...,14,02,2,1F,2025-10-04 00:15:32.000,2025-10-04 00:18:03.000,10.05,16.70,2.33,3.05
2,L8028UUD,Receiving,7655093,20,FCL,DRY,DFSU3189714,2025-10-04 00:01:37.000,2025-10-04 00:02:19.000,NaN,...,103,01,1,7B,NaN,2025-10-04 01:07:51.000,NaN,66.23,NaN,NaN
3,B9582UWW,Receiving,7654518,40,FCL,DRY,TRHU7322965,2025-10-04 00:02:09.000,2025-10-04 00:02:49.000,2025-10-04 00:12:25.000,...,38,06,5,5G,2025-10-04 00:25:39.000,2025-10-04 00:27:51.000,12.42,25.70,9.60,10.27
4,B9432PEH,Receiving,7656975,20,FCL,DRY,EMCU3974602,2025-10-04 00:02:28.000,2025-10-04 00:03:04.000,2025-10-04 00:13:42.000,...,73,03,4,1A,2025-10-04 00:30:46.000,2025-10-04 00:33:16.000,13.67,30.80,10.63,11.23
5,H1948GA,Receiving,7645325,20,MTY,DRY,TEGU2910118,2025-10-04 00:02:33.000,2025-10-04 00:03:04.000,2025-10-04 00:04:51.000,...,19,07,2,4B,NaN,2025-10-04 00:19:52.000,8.67,17.32,1.78,2.30
6,H8123OQ,Receiving,7654817,40,FCL,DRY,TXGU8462615,2025-10-04 00:03:00.000,2025-10-04 00:03:34.000,2025-10-04 00:11:37.000,...,02,02,5,5G,2025-10-04 00:23:13.000,2025-10-04 00:25:47.000,10.02,22.78,8.05,8.62
7,H1948GA,Receiving,7645317,20,MTY,DRY,TEGU2963170,2025-10-04 00:03:15.000,2025-10-04 00:03:49.000,2025-10-04 00:04:51.000,...,19,07,3,4B,NaN,2025-10-04 00:19:52.000,11.13,16.62,1.03,1.60
8,H9704OW,Receiving,7654602,20,FCL,DRY,EITU0546891,2025-10-04 00:03:27.000,2025-10-04 00:04:04.000,2025-10-04 00:13:22.000,...,71,01,1,1A,2025-10-04 00:28:04.000,2025-10-04 00:30:35.000,10.57,27.13,9.30,9.92
9,H8874OH,Receiving,7654672,20,FCL,DRY,MRTU2156184,2025-10-04 00:03:28.000,2025-10-04 00:04:04.000,NaN,...,23,06,3,7B,NaN,2025-10-04 00:36:19.000,NaN,32.85,NaN,NaN


In [4]:
# Data info
print("\n" + "=" * 80)
print("DATA INFO")
print("=" * 80)
df.info()


DATA INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122540 entries, 0 to 122539
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   TRUCK_ID          122540 non-null  object 
 1   JOB_TYPE          122540 non-null  object 
 2   CONTAINER_KEY     122540 non-null  int64  
 3   CONTAINER_SIZE    122540 non-null  int64  
 4   CTR_STATUS        122540 non-null  object 
 5   CONTAINER_TYPE    122540 non-null  object 
 6   CONTAINER_NO      122540 non-null  object 
 7   GATE_IN           122540 non-null  object 
 8   PARKING_IN        122540 non-null  object 
 9   PARKING_OUT       106007 non-null  object 
 10  BUFFER_IN         83465 non-null   object 
 11  STACK_TIME        122540 non-null  object 
 12  SLOT              122539 non-null  object 
 13  ROW               122539 non-null  object 
 14  TIER              122539 non-null  object 
 15  STACK             122539 non-null  object 
 16  BUFFER_OU

## 3. Filter: Remove BLOK 7

**Important:** Blok 7 (7A, 7B, 7C, dll) tidak diprediksi sesuai requirement

In [5]:
block_col = 'STACK' if 'STACK' in df.columns else 'BLOCK'

print("=" * 80)
print(f"BLOCK DISTRIBUTION (Column: {block_col})")
print("=" * 80)

# Show all unique blocks
blocks = df[block_col].value_counts().sort_index()
print(f"\nTotal unique blocks: {len(blocks)}")
print(f"\nBlock distribution:")
print(blocks)

# Check blok 7
block7_count = df[df[block_col].astype(str).str.startswith('7')].shape[0]
print(f"\n⚠️  Records from BLOK 7: {block7_count:,} ({block7_count/len(df)*100:.2f}%)")

BLOCK DISTRIBUTION (Column: STACK)

Total unique blocks: 45

Block distribution:
STACK
1A     6339
1B     4891
1C     6302
1D     7571
1E     7339
1F     8120
1G     7465
1Z      119
2A     2634
2B     2300
2C     1822
2Z      957
3A       37
3B       54
3C       78
3D       14
3Z      358
4A     6318
4B     5247
5A     1915
5B     4543
5C     5231
5D     5829
5E     5991
5F     5996
5G     5601
6A      921
6B     1731
6C     1536
6D       10
6Z      166
7A     6026
7B     7261
9        16
CFS      72
D1      797
EXT       4
L&F      12
LR1      49
LR2      75
LRJ       1
LRK      15
S03     102
SSD     667
SSS       7
Name: count, dtype: int64

⚠️  Records from BLOK 7: 13,287 (10.84%)


In [6]:
# Remove BLOK 7
print("=" * 80)
print("FILTERING: REMOVE BLOK 7")
print("=" * 80)

before_count = len(df)

# Filter: keep only non-block-7
df = df[~df[block_col].astype(str).str.startswith('7')].copy()

after_count = len(df)
removed_count = before_count - after_count

print(f"\n✅ Filtering complete:")
print(f"   Records before: {before_count:,}")
print(f"   Records after:  {after_count:,}")
print(f"   Removed:        {removed_count:,} ({removed_count/before_count*100:.2f}%)")
print(f"\n   Remaining blocks:")
remaining_blocks = df[block_col].value_counts().sort_index()
print(remaining_blocks)

FILTERING: REMOVE BLOK 7

✅ Filtering complete:
   Records before: 122,540
   Records after:  109,253
   Removed:        13,287 (10.84%)

   Remaining blocks:
STACK
1A     6339
1B     4891
1C     6302
1D     7571
1E     7339
1F     8120
1G     7465
1Z      119
2A     2634
2B     2300
2C     1822
2Z      957
3A       37
3B       54
3C       78
3D       14
3Z      358
4A     6318
4B     5247
5A     1915
5B     4543
5C     5231
5D     5829
5E     5991
5F     5996
5G     5601
6A      921
6B     1731
6C     1536
6D       10
6Z      166
9        16
CFS      72
D1      797
EXT       4
L&F      12
LR1      49
LR2      75
LRJ       1
LRK      15
S03     102
SSD     667
SSS       7
Name: count, dtype: int64


## 4. Validate SLOT, ROW, TIER

In [7]:
print("=" * 80)
print("VALIDATION: SLOT, ROW, TIER")
print("=" * 80)

# Check if columns exist
location_cols = ['SLOT', 'ROW', 'TIER']
missing_cols = [col for col in location_cols if col not in df.columns]

if missing_cols:
    print(f"\n⚠️  Missing columns: {missing_cols}")
    print(f"   Available columns: {df.columns.tolist()}")
    print(f"   Please check column names!")
else:
    print("\n✅ All location columns found!")
    
    # ============================================================================
    # CONVERT TO NUMERIC FIRST (handle non-numeric values)
    # ============================================================================
    print(f"\n🔄 Converting to numeric...")
    
    for col in location_cols:
        print(f"\n{col}:")
        print(f"  Original type: {df[col].dtype}")
        
        # Convert to numeric (coerce errors to NaN)
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
        print(f"  New type: {df[col].dtype}")
        print(f"  Min: {df[col].min()}")
        print(f"  Max: {df[col].max()}")
        print(f"  Unique values: {df[col].nunique()}")
        print(f"  Missing/NaN values: {df[col].isnull().sum()}")
    
    # Remove rows with NaN in location columns
    nan_mask = df[location_cols].isnull().any(axis=1)
    if nan_mask.sum() > 0:
        print(f"\n⚠️  Found {nan_mask.sum():,} rows with non-numeric location data")
        print(f"   Removing these rows...")
        df = df[~nan_mask].copy()
        print(f"   Remaining records: {len(df):,}")
    
    # ============================================================================
    # VALIDATE RANGES
    # ============================================================================
    print("\n" + "=" * 80)
    print("RANGE VALIDATION")
    print("=" * 80)
    
    invalid_mask = pd.Series([False] * len(df), index=df.index)
    
    # SLOT: typically 1-150 (allow up to 200 for safety)
    # Note: 102-103 di blok 3Z adalah valid!
    invalid_slot = (df['SLOT'] < 0) | (df['SLOT'] > 200)
    if invalid_slot.sum() > 0:
        print(f"\n⚠️  Invalid SLOT (< 0 or > 200): {invalid_slot.sum():,} records")
        print(f"   Sample invalid slots: {df[invalid_slot]['SLOT'].unique()[:10].tolist()}")
        invalid_mask |= invalid_slot
    else:
        print(f"\n✅ SLOT: All values valid (0-200)")
        print(f"   Range: {int(df['SLOT'].min())} - {int(df['SLOT'].max())}")
    
    # ROW: typically 1-20 (allow up to 50)
    invalid_row = (df['ROW'] < 0) | (df['ROW'] > 50)
    if invalid_row.sum() > 0:
        print(f"\n⚠️  Invalid ROW (< 0 or > 50): {invalid_row.sum():,} records")
        print(f"   Sample invalid rows: {df[invalid_row]['ROW'].unique()[:10].tolist()}")
        invalid_mask |= invalid_row
    else:
        print(f"\n✅ ROW: All values valid (0-50)")
        print(f"   Range: {int(df['ROW'].min())} - {int(df['ROW'].max())}")
    
    # TIER: typically 1-7 (allow up to 9)
    invalid_tier = (df['TIER'] < 0) | (df['TIER'] > 9)
    if invalid_tier.sum() > 0:
        print(f"\n⚠️  Invalid TIER (< 0 or > 9): {invalid_tier.sum():,} records")
        print(f"   Sample invalid tiers: {df[invalid_tier]['TIER'].unique()[:10].tolist()}")
        invalid_mask |= invalid_tier
    else:
        print(f"\n✅ TIER: All values valid (0-9)")
        print(f"   Range: {int(df['TIER'].min())} - {int(df['TIER'].max())}")
    
    # Remove invalid records
    if invalid_mask.sum() > 0:
        print(f"\n❌ Removing {invalid_mask.sum():,} records with invalid location data...")
        df = df[~invalid_mask].copy()
        print(f"   Remaining records: {len(df):,}")
    else:
        print(f"\n✅ All location data valid!")

VALIDATION: SLOT, ROW, TIER

✅ All location columns found!

🔄 Converting to numeric...

SLOT:
  Original type: object
  New type: float64
  Min: 1.0
  Max: 102.0
  Unique values: 101
  Missing/NaN values: 3258

ROW:
  Original type: object
  New type: float64
  Min: 1.0
  Max: 13.0
  Unique values: 13
  Missing/NaN values: 3258

TIER:
  Original type: object
  New type: float64
  Min: 1.0
  Max: 7.0
  Unique values: 7
  Missing/NaN values: 3258

⚠️  Found 3,258 rows with non-numeric location data
   Removing these rows...
   Remaining records: 105,995

RANGE VALIDATION

✅ SLOT: All values valid (0-200)
   Range: 1 - 102

✅ ROW: All values valid (0-50)
   Range: 1 - 13

✅ TIER: All values valid (0-9)
   Range: 1 - 7

✅ All location data valid!


## 5. Handle Missing Values & Duplicates

In [8]:
print("=" * 80)
print("MISSING VALUES CHECK")
print("=" * 80)

missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing': missing.values,
    'Percentage': missing_pct.values
})
missing_df = missing_df[missing_df['Missing'] > 0].sort_values('Missing', ascending=False)

if len(missing_df) > 0:
    print(f"\n⚠️  Columns with missing values:")
    print(missing_df.to_string(index=False))
else:
    print(f"\n✅ No missing values!")

MISSING VALUES CHECK

⚠️  Columns with missing values:
          Column  Missing  Percentage
      BUFFER_OUT    26659       25.15
       BUFFER_IN    23252       21.94
     PARKING_OUT     2797        2.64
   GATE_IN_STACK     2797        2.64
     PARK_IN_OUT     2797        2.64
GATE_IN_PARK_OUT     2797        2.64


In [9]:
print("\n" + "=" * 80)
print("DUPLICATE CHECK")
print("=" * 80)

duplicates = df.duplicated().sum()
if duplicates > 0:
    print(f"\n⚠️  Found {duplicates:,} duplicate records")
    print(f"   Removing duplicates...")
    df = df.drop_duplicates().copy()
    print(f"   Remaining records: {len(df):,}")
else:
    print(f"\n✅ No duplicates found!")


DUPLICATE CHECK

✅ No duplicates found!


## 6. Final Statistics

In [10]:
print("=" * 80)
print("FINAL DATASET STATISTICS")
print("=" * 80)

print(f"\n📊 Dataset Summary:")
print(f"   Total records: {len(df):,}")
print(f"   Total columns: {df.shape[1]}")

if 'SLOT' in df.columns:
    print(f"\n📍 Location Summary:")
    print(f"   Unique slots: {df['SLOT'].nunique()}")
    print(f"   Slot range: {df['SLOT'].min()} - {df['SLOT'].max()}")
    print(f"   Unique rows: {df['ROW'].nunique()}")
    print(f"   Row range: {df['ROW'].min()} - {df['ROW'].max()}")
    print(f"   Unique tiers: {df['TIER'].nunique()}")
    print(f"   Tier range: {df['TIER'].min()} - {df['TIER'].max()}")

print(f"\n🏢 Block Distribution:")
block_dist = df[block_col].value_counts().sort_index()
print(block_dist)
print(f"\n   Total blocks: {len(block_dist)}")
print(f"   Blocks included: {', '.join(sorted(block_dist.index.astype(str).unique()))}")
print(f"   ✅ Blok 7 excluded!")

# Check if target column exists
target_col = 'GATE_IN_STACK' if 'GATE_IN_STACK' in df.columns else None
if target_col:
    print(f"\n🎯 Target Variable ({target_col}):")
    print(f"   Mean: {df[target_col].mean():.2f} minutes")
    print(f"   Median: {df[target_col].median():.2f} minutes")
    print(f"   Std: {df[target_col].std():.2f} minutes")
    print(f"   Min: {df[target_col].min():.2f} minutes")
    print(f"   Max: {df[target_col].max():.2f} minutes")

FINAL DATASET STATISTICS

📊 Dataset Summary:
   Total records: 105,995
   Total columns: 22

📍 Location Summary:
   Unique slots: 101
   Slot range: 1.0 - 102.0
   Unique rows: 13
   Row range: 1.0 - 13.0
   Unique tiers: 7
   Tier range: 1.0 - 7.0

🏢 Block Distribution:
STACK
1A     6339
1B     4891
1C     6302
1D     7571
1E     7339
1F     8120
1G     7465
1Z      119
2A     2634
2B     2300
2C     1822
3Z      358
4A     6318
4B     5247
5A     1915
5B     4543
5C     5231
5D     5829
5E     5991
5F     5996
5G     5601
6B     1731
6C     1536
D1      797
Name: count, dtype: int64

   Total blocks: 24
   Blocks included: 1A , 1B , 1C , 1D , 1E , 1F , 1G , 1Z , 2A , 2B , 2C , 3Z , 4A , 4B , 5A , 5B , 5C , 5D , 5E , 5F , 5G , 6B , 6C , D1 
   ✅ Blok 7 excluded!

🎯 Target Variable (GATE_IN_STACK):
   Mean: 17.50 minutes
   Median: 13.32 minutes
   Std: 14.12 minutes
   Min: -378.53 minutes
   Max: 521.85 minutes


## 7. Save Cleaned Dataset

In [11]:
print("\n" + "=" * 80)
print("SAVING CLEANED DATASET")
print("=" * 80)

# Create output directory
output_path = Path('../Data/processed/dataset_rapi_2bulan.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)

# Save to CSV
df.to_csv(output_path, index=False)

file_size_mb = output_path.stat().st_size / (1024 * 1024)

print(f"\n✅ Dataset saved successfully!")
print(f"   Path: {output_path}")
print(f"   Size: {file_size_mb:.2f} MB")
print(f"   Records: {len(df):,}")
print(f"   Columns: {df.shape[1]}")
print(f"\n🎯 Ready for feature engineering!")


SAVING CLEANED DATASET

✅ Dataset saved successfully!
   Path: ..\Data\processed\dataset_rapi_2bulan.csv
   Size: 25.31 MB
   Records: 105,995
   Columns: 22

🎯 Ready for feature engineering!


## 8. Cleaning Summary

In [12]:
print("=" * 80)
print("CLEANING SUMMARY")
print("=" * 80)

print("\n✅ Cleaning steps completed:")
print("   1. ✅ Loaded dataset (kolom SLOT/ROW/TIER terpisah)")
print("   2. ✅ Filtered BLOK 7 (tidak diprediksi)")
print("   3. ✅ Validated SLOT/ROW/TIER ranges")
print("   4. ✅ Removed invalid records")
print("   5. ✅ Removed duplicates")
print("   6. ✅ Saved clean dataset")

print("\n📝 Next steps:")
print("   → Run eda_feature_engineering2bulan_FIXED.ipynb")
print("   → Model akan otomatis correct karena data sudah bersih!")
print("\n" + "=" * 80)

CLEANING SUMMARY

✅ Cleaning steps completed:
   1. ✅ Loaded dataset (kolom SLOT/ROW/TIER terpisah)
   2. ✅ Filtered BLOK 7 (tidak diprediksi)
   3. ✅ Validated SLOT/ROW/TIER ranges
   4. ✅ Removed invalid records
   5. ✅ Removed duplicates
   6. ✅ Saved clean dataset

📝 Next steps:
   → Run eda_feature_engineering2bulan_FIXED.ipynb
   → Model akan otomatis correct karena data sudah bersih!

